# Practical: Automated Feedback from Physiological Data for Time Perception
Welcome to the practical on automated feedback from physiological data for time perception.
In this practical you will learn about the following 4 topics:
1. Data cleaning: Removing noise and artifacts from the raw data.
2. Feature extraction: Identifying relevant features from the cleaned data that are indicative of time perception.
3. Classification: Using machine learning algorithms to classify data based on extracted features.
4. Additional steps: Exploring advanced techniques such as feature selection methods, feature importance measurements, and automated machine learning (AutoML).

We will use the same data as Aust et al. 2024 (https://arxiv.org/abs/2404.15213). For simplicity, we will only use electrodermal activity (EDA) data.
So without further ado, let's get started!
First you will import required libraries.

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install neurokit2
import neurokit2 as nk

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

!pip install shap
import shap
from sklearn.feature_selection import RFECV, SequentialFeatureSelector

!pip install naiveautoml
import naiveautoml
import logging

# connect your drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Data Cleaning
Great! Now we have all required libraries imported. Next, we will load the data.

In [ ]:
# load the raw EDA data
raw_eda = ...

# display the overall shape and the column names of the data
...

# display the first few rows of the data
...

Okay, the data consists of a `LocalTimestamp` and `EA` that is the raw EDA data. Next, we will plot the raw EDA data.

In [ ]:
# plot the raw EDA data
plt.figure(figsize=(15, 5))
...
plt.xlabel('Time')
plt.ylabel('EDA')
plt.title('Raw EDA data')

You may have noticed that the time axis does not tell you anything. To make it more informative, we will convert the `LocalTimestamp` to a more readable format. We can do this by subtracting the first timestamp from all timestamps such that each experiment starts at 0.

In [ ]:
# subtract the first time stamp from all other time stamps an plot the data again
raw_eda['LocalTimestamp'] = ...

# plot the raw EDA data
plt.figure(figsize=(15, 5))
...
plt.xlabel('Time')
plt.ylabel('EDA')
plt.title('Raw EDA data')

Now the time make sense, it seems that the experiment lasted for about 180 seconds or 3 minutes. Next, we will clean the data to remove noise and artifacts. Therefore, we use the NeuroKit library we imported earlier. Before we can clean the data we need to know the sampling rate of our data because this information is a parameter we need to hand over to the cleaning method.

In [ ]:
# calculate the sampling rate of the eda data in Hz
sampling_rate = ...
sampling_rate_int = ...

print(f"Sampling rate: {sampling_rate} Hz")
print(f"Rounded sampling rate: {sampling_rate_int} Hz")

# clean the data using the NeuroKit library
#  You can check out the documentation here: https://neuropsychology.github.io/NeuroKit/functions/eda.html
#  You can use report="subject_1_report.html" to print a visual overview of the cleaning process
cleaned_eda_signals, info = ...

The `cleaned_eda_signals` contain the cleaned EDA data. `info` contains information of each SCR peak and sampling frequency.
Now, we can plot the individual EDA components, needed for calculating the features.

In [ ]:
# plot the individual EDA components
plt.figure(figsize=(15, 5))
...
plt.legend()

## 2. Feature Extraction
Next, we can use the cleaned EDA data to extract features. We will use the following features: SCR peaks, SCR peak amplitude mean, EDA tonic standard deviation, EDA sympathetic, EDA sympathetic N, EDA autocorrelation. The NeuroKit library provides a function to extract these features.

In [81]:
# extract features from the cleaned EDA data
#  method can be "interval-related" or "event-related" (event-related is for sequences under 10 seconds) and we are using an interval and not events
eda_features = ...

Great, now we have extracted the EDA features. Now we need to get the subjective time perception of the participants.
We start by loading the questionnaire answers of all participants.

In [ ]:
# load the questionnaire answers
questionnaire = ...

# display the first few rows of the data
...

The important columns are `ParticipantID` and `Session` to assign the corresponding answer to the setting and participant. Further, the column `PassageOfTimeSlowFast` provides us with the answer of slow or fast subjective time perception.
So now we can extract the answer for the first participant and first session.
(The mapping is 1: very slow, 2: slow, 3: normal, 4: fast, 5: very fast)

In [ ]:
# select the subjective time perception of participant 1 in session 1
subjective_time_perception_label = ...

# print the subjective time perception
...

Now you learned how you can get the features and the appropriate labels. In the next step you will automate the process for all participants and sessions.

In [84]:
# define the participants
participants = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# define the sessions
sessions = [1, 2, 3, 4]

# final Pandas dataframe that holds all data
all_data = pd.DataFrame(columns=["participant", "session", "SCR_Peaks_N",
                    "SCR_Peaks_Amplitude_Mean",
                    "EDA_Tonic_SD",
                    "EDA_Sympathetic",
                    "EDA_SympatheticN",
                    "EDA_Autocorrelation", "subjective_time_perception"])
counter = 0

# iterate over all participants
for p in participants:
    # iterate over all sessions
    for s in sessions:
        # load the raw EDA data
        ...
        # subtract the first time stamp from all other time stamps
        ...
        # calculate the sampling rate of the eda data in Hz
        ...

        # clean the data using the NeuroKit library
        cleaned_eda_signals, info = ...

        # extract features from the cleaned EDA data
        eda_features = ...

        # select the subjective time perception of the participant in the session
        subjective_time_perception_label = ...

        # append the data to the final dataframe
        all_data.loc[counter] = [p, s, float(eda_features["SCR_Peaks_N"].iloc[0]), float(eda_features["SCR_Peaks_Amplitude_Mean"].iloc[0]), float(eda_features["EDA_Tonic_SD"].iloc[0]), float(eda_features["EDA_Sympathetic"].iloc[0]), float(eda_features["EDA_SympatheticN"].iloc[0]), float(eda_features["EDA_Autocorrelation"].iloc[0]), subjective_time_perception_label]
        counter += 1

Great! Now you have all the data in one place. Next, we will use machine learning to classify the data based on the extracted features.
Therefore, we first split the data into training and test data. We will use 80% of the data for training and 20% for testing.

In [ ]:
# split the data into training and testing data (80% training, 20% testing)
#  random_state is set to 42 to ensure reproducibility
x_train, x_test, y_train, y_test = ...

print(f"X train shape: {x_train.shape}, X test shape: {x_test.shape}, y train shape: {y_train.shape}, y test shape: {y_test.shape}")

## 3. Classification
Now we can use the training data to train our classifier and evaluate it on the test data.

In [ ]:
# create classifier
#  see: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html for more information and hyperparameter selection
dtc = ...

# train the classifier
...

# evaluate the classifier
#  predict the labels of the test data
y_pred = ...
#  predict the probabilities of the test data
y_pred_proba = ...

# calculate the accuracy of the classifier
print(f"Accuracy: {...}")
# calculate the F1 score of the classifier
print(f"F1 score: {...}")
# calculate the ROC AUC score of the classifier
print(f"ROC AUC score: {...}")

## 4. Additional steps
Great! You have successfully trained and evaluated a classifier to predict subjective time perception based on EDA features.
However, the results are not so satisfying yet. Therefore, we will explore additional steps to improve the classification performance.
Let's start with simplifying the problem by reducing the number of classes.

In [ ]:
# reduce the number of classes to slow and fast
all_data['subjective_time_perception_reduced'] = ...

# let's see the distribution of the classes before and after
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
...
plt.title('Subjective time perception distribution')
plt.xlabel('Subjective time perception')
plt.ylabel('Frequency')
plt.subplot(1, 2, 2)
...
plt.title('Reduced subjective time perception distribution')
plt.xlabel('Subjective time perception')
plt.ylabel('Frequency')

This simplified the problem. Let's train and evaluate the classifier again. We will try a different classifier this time: Random Forest (ensemble of decision trees).

In [ ]:
# use the reduced label
x_train, x_test, y_train, y_test = ...

print(f"X train shape: {x_train.shape}, X test shape: {x_test.shape}, y train shape: {y_train.shape}, y test shape: {y_test.shape}")


rfc = ...

# train the classifier
...

# evaluate the classifier
#  predict the labels of the test data
...

# calculate the accuracy of the classifier
print(f"Accuracy: {...}")
# calculate the F1 score of the classifier
print(f"F1 score: {...}")
# calculate the ROC AUC score of the classifier
print(f"ROC AUC score: {...)}")

Great! The classification performance improved. Next, we will explore feature importance to identify the most relevant features.
Therefore, we will use shape-based feature importance.

In [ ]:
# create the explainer
explainer = ...
# calculate the shap values based on the test data
shap_value = ...
# returns probability for class 0 and 1, but we only need one bc p = 1 - p
shap_value.values = shap_value.values[:, :, 1]
shap_value.base_values = shap_value.base_values[:, 1]

# plot the feature importance
plt.figure()
shap.plots.bar(shap_value, max_display=..., show=True)

Seeing that some features are more important than others, we can try to remove the less important features and retrain the classifier.

In [ ]:
# create a sequential feature selector
selector = ...
# fit the selector based on the training data
x_train = ...
# transform the test data accordingly
x_test = ...

# store results
result_df = pd.DataFrame(zip(["SCR_Peaks_N", "SCR_Peaks_Amplitude_Mean", "EDA_Tonic_SD", "EDA_Sympathetic", "EDA_SympatheticN", "EDA_Autocorrelation"], selector.get_support()))
result_df.columns = ["features", "used"]

# print the results
print(result_df)

# train the classifier
...

# evaluate the classifier
#  predict the labels of the test data
...

# calculate the accuracy of the classifier
print(f"Accuracy: {...}")
# calculate the F1 score of the classifier
print(f"F1 score: {...}")
# calculate the ROC AUC score of the classifier
print(f"ROC AUC score: {...}")

Finally, we can have a look at automated model selection, which can be helpful to find the correct hyperparameters for the classifier.

In [ ]:
# do logging
logger = logging.getLogger('naiveautoml')
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

# create the NaiveAutoML object
naml = ...

# fit the NaiveAutoML object
...

# print the best model
print("---------------------------------")
print(naml.chosen_model)
print("---------------------------------")

In [ ]:
# Testing the findings from the AutoML
svc = ...

# fit classifier
...

# predict classifier labels
y_pred = ...

# calculate the accuracy of the classifier
print(f"Accuracy: {...}")

### Conclusion
Now you have learned the basics of processing and classifying physiological data for time perception. You have learned how to clean the data, extract features, classify the data, and explore additional steps to improve the classification performance.

It is time to explore the data on your own. You can try different classifiers, feature selection methods, and hyperparameter optimization techniques to improve the classification performance further. You can also try different features or even combine features from different physiological data sources to improve the classification performance.

You can also play around with the sequences provided by the `timestamps.csv`.

In [ ]:
# TODO play around!